In [1]:
import os

import joblib
from azureml.core import Dataset, Environment, Experiment, Workspace, ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import Pipeline, PipelineEndpoint, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration

In [7]:
ws = Workspace.from_config()

aml_compute_target = "clusteraml"

try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("Found existing compute target: {}".format(aml_compute_target))
except ComputeTargetException:
    print("Creating new compute target: {}".format(aml_compute_target))
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 1)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)


Found existing compute target: clusteraml


In [13]:
CustomerInference_dataset = Dataset.get_by_name(ws, name='customer_tabular')
CustomerInference_pipeline_param = PipelineParameter(name="CustomerInference_pipeline_param", default_value=CustomerInference_dataset)
CustomerInference_ds_consumption = DatasetConsumptionConfig("CustomerInference_dataset", CustomerInference_pipeline_param)

OutputPathParameter = PipelineParameter(name="output_path", default_value="CustomerIncomeOutput/CustomerIncomeOutput.csv")
OutputDatastoreParameter = PipelineParameter(name="output_datastore", default_value="workspaceblobstore")

env = Environment.from_conda_specification(name = 'env', file_path = './CustomerIncomeInferencePipeline.yml')

run_config = RunConfiguration()
run_config.environment = env

scoring_step = PythonScriptStep(
    name="Scoring_Step",
    script_name='score.py',
    arguments=["--input_data", CustomerInference_ds_consumption, "--output_path", OutputPathParameter, "--output_datastore", OutputDatastoreParameter],
    inputs=[CustomerInference_ds_consumption],
    compute_target=aml_compute_target,
    source_directory='./scripts',
    runconfig = run_config,
    allow_reuse=False)

print("Scoring step created")

pipeline = Pipeline(workspace=ws, steps=[scoring_step])

print("Pipeline is built")

Scoring step created
Pipeline is built


In [14]:
pipeline_endpoint_name = 'IncomeEndpoint14'
experiment_name = 'ci-aml-custom-models'

pipeline_endpoint = PipelineEndpoint.publish(workspace=ws, name=pipeline_endpoint_name, pipeline=pipeline, description="Income Endpoint")
published_pipeline = pipeline.publish(name=pipeline_endpoint_name, description="published pipeline")
pipeline_endpoint = PipelineEndpoint.get(workspace=ws, name=pipeline_endpoint_name) 
pipeline_endpoint.add_default(pipeline=published_pipeline)
published_pipeline

Created step Scoring_Step [d0b2d337][7f9f492d-8746-4d18-8a0d-0af947dd52fa], (This step will run and generate new outputs)


Name,Id,Status,Endpoint
IncomeEndpoint14,cd98b585-ea1a-41fb-8fee-e2a0290b1d1d,Active,REST Endpoint


In [15]:
experiment = Experiment(ws, experiment_name)

pipeline_run = experiment.submit(pipeline_endpoint)

pipeline_run.get_details()

{'runId': 'c28de5ca-b54f-462a-9819-3cf74bf0e00e',
 'status': 'Preparing',
 'services': {},
 'properties': {'azureml.runsource': 'azureml.PipelineRun',
  'runSource': 'SDK',
  'runType': 'SDK',
  'azureml.parameters': '{"output_path":"CustomerIncomeOutput/CustomerIncomeOutput.csv","output_datastore":"workspaceblobstore"}',
  'azureml.continue_on_step_failure': 'False',
  'azureml.pipelineComponent': 'pipelinerun',
  'azureml.pipelineid': 'cd98b585-ea1a-41fb-8fee-e2a0290b1d1d'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {},
 'submittedBy': 'Estera Kot'}

In [16]:
pipeline_run.wait_for_completion(show_output=True)


PipelineRunId: c28de5ca-b54f-462a-9819-3cf74bf0e00e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/c28de5ca-b54f-462a-9819-3cf74bf0e00e?wsid=/subscriptions/202fb1eb-7a41-4c55-8a8d-7f47e105246c/resourcegroups/ci_aml_custom_model/workspaces/ci_aml_main_workspace&tid=5e5d1d17-c4b2-4b93-8e7f-1370db4f1d6c
PipelineRun Status: Running


StepRunId: b0d60e45-6934-4413-86ff-a7caa49df6b8
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/b0d60e45-6934-4413-86ff-a7caa49df6b8?wsid=/subscriptions/202fb1eb-7a41-4c55-8a8d-7f47e105246c/resourcegroups/ci_aml_custom_model/workspaces/ci_aml_main_workspace&tid=5e5d1d17-c4b2-4b93-8e7f-1370db4f1d6c
StepRun( Scoring_Step ) Status: Running

StepRun(Scoring_Step) Execution Summary
StepRun( Scoring_Step ) Status: Finished

Warnings:
This run might be using a new job runtime with improved performance and error reporting. The logs from your script are in user_logs/std_log.txt. Please let us know if you run into any issues, and if yo

'Finished'